In [9]:
import cv2
import numpy as np
from pathlib import Path
import random

In [10]:
x_train = np.load('Data/x_train.npy') 
y_train = np.load('Data/y_train.npy')
x_test = np.load('Data/x_test.npy')
y_test = np.load('Data/y_test.npy')

In [11]:
class LetterAugmentor:
    """
    Image augmentation for Arabic and English letter recognition.
    Applies various transformations while preserving letter integrity.
    """
    
    def __init__(self, output_dir='augmented_images'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
    
    def rotate(self, image, angle=None):
        """Rotate image by a random angle between -15 and 15 degrees"""
        if angle is None:
            angle = random.randint(-15, 15)
        
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, matrix, (w, h), 
                                 borderMode=cv2.BORDER_REPLICATE)
        return rotated
    
    def scale(self, image, scale_factor=None):
        """Scale image between 90% and 110%"""
        if scale_factor is None:
            scale_factor = random.randint(0.9, 1.1)
        
        h, w = image.shape[:2]
        new_h, new_w = int(h * scale_factor), int(w * scale_factor)
        scaled = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        
        # Pad or crop to maintain original size
        if scaled.shape[0] < h or scaled.shape[1] < w:
            # Pad if smaller
            pad_h = max(0, h - scaled.shape[0])
            pad_w = max(0, w - scaled.shape[1])
            scaled = cv2.copyMakeBorder(scaled, pad_h//2, pad_h - pad_h//2,
                                       pad_w//2, pad_w - pad_w//2,
                                       cv2.BORDER_REPLICATE)
        else:
            # Crop if larger
            start_h = (scaled.shape[0] - h) // 2
            start_w = (scaled.shape[1] - w) // 2
            scaled = scaled[start_h:start_h+h, start_w:start_w+w]
        
        return scaled
    
    def translate(self, image, shift_x=None, shift_y=None):
        """Shift image by random pixels (±10% of dimensions)"""
        h, w = image.shape[:2]
        
        if shift_x is None:
            shift_x = random.randint(-int(w*0.1), int(w*0.1))
        if shift_y is None:
            shift_y = random.randint(-int(h*0.1), int(h*0.1))
        
        matrix = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
        translated = cv2.warpAffine(image, matrix, (w, h),
                                    borderMode=cv2.BORDER_REPLICATE)
        return translated
    
    def add_noise(self, image, noise_type='gaussian'):
        """Add Gaussian or salt-and-pepper noise"""
        if noise_type == 'gaussian':
            mean = 0
            sigma = random.randint(5, 15)
            gauss = np.random.normal(mean, sigma, image.shape).astype(np.float32)
            noisy = np.clip(image.astype(np.float32) + gauss, 0, 255).astype(np.uint8)
            return noisy
        
        elif noise_type == 'salt_pepper':
            prob = random.uniform(0.01, 0.03)
            noisy = image.copy()
            
            # Salt
            salt = np.random.random(image.shape[:2]) < prob/2
            noisy[salt] = 255
            
            # Pepper
            pepper = np.random.random(image.shape[:2]) < prob/2
            noisy[pepper] = 0
            
            return noisy
        
        return image
    
    def adjust_brightness_contrast(self, image, alpha=None, beta=None):
        """
        Adjust brightness and contrast
        alpha: contrast control (1.0-3.0)
        beta: brightness control (-50 to 50)
        """
        if alpha is None:
            alpha = random.randint(0.8, 1.2)
        if beta is None:
            beta = random.randint(-30, 30)
        
        adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        return adjusted
    
    def blur(self, image, kernel_size=None):
        """Apply Gaussian blur"""
        if kernel_size is None:
            kernel_size = random.choice([3, 5])
        
        blurred = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
        return blurred
    
    def elastic_transform(self, image, alpha=None, sigma=None):
        """Apply elastic deformation for handwriting variation"""
        if alpha is None:
            alpha = random.randint(5, 15)
        if sigma is None:
            sigma = random.randint(3, 5)
        
        h, w = image.shape[:2]
        
        # Generate random displacement fields
        dx = cv2.GaussianBlur((np.random.rand(h, w) * 2 - 1), (0, 0), sigma) * alpha
        dy = cv2.GaussianBlur((np.random.rand(h, w) * 2 - 1), (0, 0), sigma) * alpha
        
        # Create meshgrid
        x, y = np.meshgrid(np.arange(w), np.arange(h))
        x = np.clip(x + dx, 0, w-1).astype(np.float32)
        y = np.clip(y + dy, 0, h-1).astype(np.float32)
        
        # Apply transformation
        transformed = cv2.remap(image, x, y, interpolation=cv2.INTER_LINEAR,
                               borderMode=cv2.BORDER_REPLICATE)
        return transformed
    
    def augment_image(self, image, num_augmentations=5):
        """
        Apply random augmentations to create multiple variants
        Returns list of augmented images
        """
        augmented_images = []
        
        # List of augmentation functions
        augmentations = [
            self.rotate,
            self.scale,
            self.translate,
            lambda img: self.add_noise(img, 'gaussian'),
            lambda img: self.add_noise(img, 'salt_pepper'),
            self.adjust_brightness_contrast,
            self.blur,
            self.elastic_transform
        ]
        
        for i in range(num_augmentations):
            aug_img = image.copy()
            
            # Apply 2-4 random augmentations
            num_ops = random.randint(2, 4)
            selected_augs = random.sample(augmentations, num_ops)
            
            for aug_func in selected_augs:
                aug_img = aug_func(aug_img)
            
            augmented_images.append(aug_img)
        
        return augmented_images
    
    def augment_dataset(self, input_dir, num_augmentations=5):
        """
        Augment all images in a directory
        
        Args:
            input_dir: Directory containing original images
            num_augmentations: Number of augmented versions per image
        """
        input_path = Path(input_dir)
        image_files = list(input_path.glob('*.jpg')) + \
                     list(input_path.glob('*.png')) + \
                     list(input_path.glob('*.jpeg'))
        
        print(f"Found {len(image_files)} images to augment")
        
        for img_file in image_files:
            # Read image
            image = cv2.imread(str(img_file))
            
            if image is None:
                print(f"Warning: Could not read {img_file}")
                continue
            
            # Generate augmented versions
            augmented = self.augment_image(image, num_augmentations)
            
            # Save augmented images
            base_name = img_file.stem
            for idx, aug_img in enumerate(augmented):
                output_name = f"{base_name}_aug_{idx}{img_file.suffix}"
                output_path = self.output_dir / output_name
                cv2.imwrite(str(output_path), aug_img)
            
            print(f"Processed {img_file.name}: created {len(augmented)} variants")
        
        print(f"\nAugmentation complete! Images saved to {self.output_dir}")

In [12]:
class LetterAugmentor:
    """
    Image augmentation for Arabic and English character datasets.
    Applies geometric + noise transformations while maintaining readability.
    """

    def __init__(self, output_dir='augmented_images'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)

    # -------------------------
    #  Transformations
    # -------------------------
    def rotate(self, image, angle=None):
        """Rotate ±15 degrees"""
        if angle is None:
            angle = random.uniform(-15, 15)

        h, w = image.shape[:2]
        center = (w // 2, h // 2)

        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        return cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REPLICATE)

    def scale(self, image, scale_factor=None):
        """Scale between 0.9 to 1.1"""
        if scale_factor is None:
            scale_factor = random.uniform(0.9, 1.1)

        h, w = image.shape[:2]
        new_h, new_w = int(h * scale_factor), int(w * scale_factor)

        scaled = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        # Pad or crop back
        if new_h < h or new_w < w:
            pad_h = h - new_h
            pad_w = w - new_w
            scaled = cv2.copyMakeBorder(
                scaled,
                pad_h // 2, pad_h - pad_h // 2,
                pad_w // 2, pad_w - pad_w // 2,
                cv2.BORDER_REPLICATE
            )
        else:
            start_h = (new_h - h) // 2
            start_w = (new_w - w) // 2
            scaled = scaled[start_h:start_h + h, start_w:start_w + w]

        return scaled

    def translate(self, image, shift_x=None, shift_y=None):
        """Shift by 10%"""
        h, w = image.shape[:2]
        if shift_x is None:
            shift_x = random.randint(-w // 10, w // 10)
        if shift_y is None:
            shift_y = random.randint(-h // 10, h // 10)

        M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
        return cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REPLICATE)

    def add_noise(self, image, noise_type='gaussian'):
        """Gaussian or salt-pepper noise"""
        if noise_type == 'gaussian':
            sigma = random.uniform(5, 15)
            noise = np.random.normal(0, sigma, image.shape).astype(np.float32)
            noisy = np.clip(image.astype(np.float32) + noise, 0, 255).astype(np.uint8)
            return noisy

        elif noise_type == 'salt_pepper':
            prob = random.uniform(0.01, 0.03)
            noisy = image.copy()

            # salt
            mask = np.random.random(image.shape[:2]) < prob / 2
            noisy[mask] = 255

            # pepper
            mask = np.random.random(image.shape[:2]) < prob / 2
            noisy[mask] = 0

            return noisy

        return image

    def adjust_brightness_contrast(self, image, alpha=None, beta=None):
        """Contrast (0.8–1.2) and brightness (-30 to 30)"""
        if alpha is None:
            alpha = random.uniform(0.8, 1.2)
        if beta is None:
            beta = random.randint(-30, 30)

        return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    def blur(self, image, kernel_size=None):
        """Gaussian blur"""
        if kernel_size is None:
            kernel_size = random.choice([3, 5])
        return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    def elastic_transform(self, image, alpha=None, sigma=None):
        """Elastic deformation for handwriting variation"""
        if alpha is None:
            alpha = random.uniform(5, 15)
        if sigma is None:
            sigma = random.uniform(3, 5)

        h, w = image.shape[:2]

        dx = cv2.GaussianBlur((np.random.rand(h, w) * 2 - 1), (0, 0), sigma) * alpha
        dy = cv2.GaussianBlur((np.random.rand(h, w) * 2 - 1), (0, 0), sigma) * alpha

        x, y = np.meshgrid(np.arange(w), np.arange(h))
        x = np.clip(x + dx, 0, w - 1).astype(np.float32)
        y = np.clip(y + dy, 0, h - 1).astype(np.float32)

        return cv2.remap(image, x, y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

    # -------------------------
    #  A full augmentation
    # -------------------------
    def augment_image(self, image, num_augmentations=5):
        """Apply 2–4 random augmentations to produce variations"""
        augmented_images = []

        transformations = [
            self.rotate,
            self.scale,
            self.translate,
            lambda img: self.add_noise(img, 'gaussian'),
            lambda img: self.add_noise(img, 'salt_pepper'),
            self.adjust_brightness_contrast,
            self.blur,
            self.elastic_transform
        ]

        for _ in range(num_augmentations):
            aug_img = image.copy()
            selected = random.sample(transformations, random.randint(2, 4))
            for transform in selected:
                aug_img = transform(aug_img)
            augmented_images.append(aug_img)

        return augmented_images


# ------------------------------------------
# Train augmentation processing
# ------------------------------------------

target_shape = x_train[0].shape
print("Target shape:", target_shape)

augmentor = LetterAugmentor()

X_train_aug = []
Y_train_aug = []

for i in range(len(x_train)):

    img = x_train[i]
    label = y_train[i]

    # Store original
    X_train_aug.append(img)
    Y_train_aug.append(label)

    # Prepare for CV (grayscale 2D uint8)
    img_aug = img.squeeze() if img.ndim == 3 else img.copy()
    img_aug = (img_aug * 255).astype(np.uint8) if img_aug.max() <= 1 else img_aug.astype(np.uint8)

    # Produce augmentations
    augmented_images = augmentor.augment_image(img_aug, num_augmentations=3)

    for aug in augmented_images:

        if aug.ndim == 3:
            aug = cv2.cvtColor(aug, cv2.COLOR_BGR2GRAY)

        if len(target_shape) == 3 and target_shape[-1] == 1:
            aug = np.expand_dims(aug, axis=-1)

        # Match type
        if np.issubdtype(x_train.dtype, np.floating):
            aug = aug.astype(np.float32) / 255.0
        else:
            aug = aug.astype(x_train.dtype)

        assert aug.shape == target_shape
        X_train_aug.append(aug)
        Y_train_aug.append(label)

    if (i + 1) % 1000 == 0:
        print(f"Processed {i+1}/{len(x_train)}")

x_train_aug = np.array(X_train_aug, dtype=x_train.dtype)
y_train_aug = np.array(Y_train_aug, dtype=y_train.dtype)

print("\nFinal augmented shapes:")
print("X:", x_train_aug.shape)
print("Y:", y_train_aug.shape)

np.save('x_train_augmented.npy', x_train_aug)
np.save('y_train_augmented.npy', y_train_aug)

print("\nAugmentation complete!")

Target shape: (28, 28, 1)
Processed 1000/25920
Processed 2000/25920
Processed 3000/25920
Processed 4000/25920
Processed 5000/25920
Processed 6000/25920
Processed 7000/25920
Processed 8000/25920
Processed 9000/25920
Processed 10000/25920
Processed 11000/25920
Processed 12000/25920
Processed 13000/25920
Processed 14000/25920
Processed 15000/25920
Processed 16000/25920
Processed 17000/25920
Processed 18000/25920
Processed 19000/25920
Processed 20000/25920
Processed 21000/25920
Processed 22000/25920
Processed 23000/25920
Processed 24000/25920
Processed 25000/25920

Final augmented shapes:
X: (103680, 28, 28, 1)
Y: (103680,)

Augmentation complete!
